In [ ]:
import math

In [ ]:
import os
all_data=open("text_dataset/processed_text_dataset.txt","r")
doc_id=0
vis={}
limit=10000
word_idf={}
doc_word_cnt=[]
for line in all_data.readlines():
    vis.clear()
    word_cnt={}
    all_words=line.split(" ")
    for word in all_words:
        if vis.get(word,0)==0:
            vis[word]=1
            word_cnt[word]=1
            word_idf[word]=word_idf.get(word,0)+1
        else:
            word_cnt[word]+=1
    doc_word_cnt.append(word_cnt)
    doc_id+=1
    print(str(doc_id)+" done")
    if doc_id==limit:
        break
    

In [ ]:
#each set top-100 normalized tf-idf
all_set_topk=[]
for i in range(len(doc_word_cnt)):
    word_cnt=doc_word_cnt[i]
    total_cnt=sum(word_cnt.values())
    tf_idf={}
    for word in word_cnt:
        tf=word_cnt[word]/total_cnt
        idf=math.log(doc_id/(word_idf[word]+1))
        tf_idf[word]=tf*idf
    sorted_tf_idf=sorted(tf_idf.items(),key=lambda x:x[1],reverse=True)
    while len(sorted_tf_idf)<100:
        sorted_tf_idf.append(("",0.0))
    top_k=sorted_tf_idf[:100]
    #min-max normalization
    min_val=min([x[1] for x in top_k])
    max_val=max([x[1] for x in top_k])
    for j in range(len(top_k)):
        word=top_k[j][0]
        val=top_k[j][1]
        norm_val=(val-min_val)/(max_val-min_val)
        top_k[j]=(word,norm_val)
    all_set_topk.append(top_k)
    

    

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# 计算平均 top-k 权重
average_topk = []
for i in range(100):
    word_sum = 0.0
    for j in range(len(all_set_topk)):
        word_sum += all_set_topk[j][i][1]
    average_topk.append(word_sum / len(all_set_topk))

x = np.arange(1, len(average_topk)+1)

# 1) 平均 rank–weight 曲线
plt.rcParams["font.family"] = "Times New Roman"
plt.figure(figsize=(4,3))
plt.plot(x, average_topk, marker='x', linestyle='-', color='b')
plt.xlabel("Rank",fontsize=16)
plt.ylabel("Average Weight",fontsize=16)
plt.title("Average Rank-Weight Curve",fontsize=16)
plt.grid(True, linestyle="--", alpha=0.6)
plt.savefig("average_rank_weight_curve.eps", dpi=300, bbox_inches='tight',format='eps')
plt.show()



In [ ]:
doc_sim={}
for i in range(100):
    for j in range(i+1,100):
        code=i*100001+j
        min_weight_sum=0
        max_weight_sum=0
        sum_cnt1=0
        sum_cnt2=0
        for word in doc_word_cnt[i]:
            sum_cnt1+=doc_word_cnt[i][word]
        for word in doc_word_cnt[j]:
            sum_cnt2+=doc_word_cnt[j][word]
        for word in doc_word_cnt[i]:
            idf=math.log2(doc_id/(word_idf[word]+1))
            tf1=doc_word_cnt[i][word]/sum_cnt1
            w1=idf*tf1
            tf2=doc_word_cnt[j].get(word,0)/sum_cnt2
            w2=tf2*idf
            min_weight_sum+=min(w1,w2)
            max_weight_sum+=max(w1,w2)
        for word in doc_word_cnt[j]:
            idf=math.log2(doc_id/(word_idf[word]+1))
            tf1=doc_word_cnt[i].get(word,0)
            if tf1>0:
                continue
            w1=idf*tf1
            tf2=doc_word_cnt[j].get(word,0)
            w2=tf2*idf
            min_weight_sum+=min(w1,w2)
            max_weight_sum+=max(w1,w2)
        doc_sim[code]=min_weight_sum/max_weight_sum

In [ ]:
topkmae=[]
topkare=[]
sumpairs=0
for k in range(20,120,20):
    summae=0
    sumare=0
    for i in range(100):
        for j in range(i+1,100):
            #use top-k words to compute similarity
            code=i*100001+j
            if doc_sim.get(code,0)==0:
                continue
            sumpairs+=1
            min_weight_sum=0
            max_weight_sum=0
            topk1={}
            topk2={}
            for word in all_set_topk[i][:k]:
                topk1[word[0]]=word[1]
            for word in all_set_topk[j][:k]:
                topk2[word[0]]=word[1]
            for word in topk1:
                min_weight_sum+=min(topk1[word],topk2.get(word,0))
                max_weight_sum+=max(topk1[word],topk2.get(word,0))
            for word in topk2:
                if word in topk1:
                    continue
                min_weight_sum+=min(topk2[word],0)
                max_weight_sum+=max(topk2[word],0)
            est=min_weight_sum/max_weight_sum
            mae=abs(est-doc_sim[code])
            are=abs(est-doc_sim[code])/doc_sim[code]
            summae+=mae
            sumare+=are
    topkmae.append(summae/sumpairs)  # 4950 is the number of unique pairs in 100 documents
    topkare.append(sumare/sumpairs)
    
                
                

In [ ]:
#draw top-k MAE curve
plt.figure(figsize=(4,3))
plt.plot(range(20,120,20), topkmae, marker='x', linestyle='-', color='r')
plt.ylim([0,0.05])
plt.xlabel("Top-k", fontsize=16)
plt.ylabel("MAE", fontsize=16)
# plt.title("Top-k MAE Curve", fontsize=16)
plt.grid(True, linestyle="--", alpha=0.6)


In [ ]:
plt.figure(figsize=(4,3))
plt.plot(range(20,120,20), topkare, marker='o', linestyle='-', color='r')
plt.xlabel("Top-k", fontsize=16)
plt.ylabel("ARE", fontsize=16)
plt.title("Top-k ARE Curve", fontsize=16)
plt.grid(True, linestyle="--", alpha=0.6)
